In [24]:
import tensorflow as tf
from tensorflow import keras

In [25]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [26]:
housing = fetch_california_housing()

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(housing.data, housing.target)

In [28]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train)

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [30]:
def build_model(n_hidden=1, n_neurons=30, lr_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=lr_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [31]:
model = build_model()

/home/gpu/pyenv/py3.8env/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 30)                270       
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))
model.evaluate(X_test, Y_test)

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 1.3894 - val_loss: 0.7430
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7071 - val_loss: 0.5982
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6210 - val_loss: 0.5462
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5670 - val_loss: 0.5088
Epoch 5/10
363/363 [==============================] - 1s 4ms/step - loss: 0.5315 - val_loss: 0.4876
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5137 - val_loss: 0.4761
Epoch 7/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5009 - val_loss: 0.4632
Epoch 8/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4914 - val_loss: 0.4551
Epoch 9/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4842 - val_loss: 0.4487
Epoch 10/10
162/162 [==============================] - 0s 2ms/step - loss: 0.5075


0.5075074434280396

### buidling model using KerasRegressor

In [34]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/tmp/ipykernel_10329/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [35]:
keras_reg.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

Epoch 1/10
363/363 [==============================] - 2s 4ms/step - loss: 1.0838 - val_loss: 0.7201
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6998 - val_loss: 0.6195
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6402 - val_loss: 0.5657
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5954 - val_loss: 0.5235
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5507 - val_loss: 0.5029
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5430 - val_loss: 0.4862
Epoch 7/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5165 - val_loss: 0.4708
Epoch 8/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5072 - val_loss: 0.4653
Epoch 9/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4922 - val_loss: 0.4578
Epoch 10/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4863 - val_loss: 0.4513

In [36]:
keras_reg.score(X_test, Y_test)

162/162 [==============================] - 0s 2ms/step - loss: 0.5073


-0.5073366761207581

### Randomized Search CV

In [37]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [38]:
reciprocal(2, 4)

In [40]:
params_distrib = {"n_hidden": [0,1,2,3,4,5],
                 "n_neurons": np.arange(1,100),
                 "lr_rate": reciprocal(3e-4, 3e-3)}

In [41]:
rnd_search_cv= RandomizedSearchCV(keras_reg, params_distrib, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, Y_train, epochs=10, validation_data=(X_valid, Y_valid))

Epoch 1/10


/home/gpu/pyenv/py3.8env/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 4ms/step - loss: 4.2303 - val_loss: 2.5828
Epoch 2/10
242/242 [==============================] - 1s 3ms/step - loss: 1.7201 - val_loss: 1.3826
Epoch 3/10
242/242 [==============================] - 1s 3ms/step - loss: 1.0285 - val_loss: 0.9854
Epoch 4/10
242/242 [==============================] - 1s 3ms/step - loss: 0.8020 - val_loss: 0.8377
Epoch 5/10
242/242 [==============================] - 1s 3ms/step - loss: 0.7194 - val_loss: 0.7723
Epoch 6/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6845 - val_loss: 0.7366
Epoch 7/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6659 - val_loss: 0.7117
Epoch 8/10
242/242 [==============================] - 1s 4ms/step - loss: 0.6529 - val_loss: 0.6916
Epoch 9/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6426 - val_loss: 0.6745
Epoch 10/10
121/121 [==============================] - 0s 2ms/step - loss: 1.3675
Epoch 1/10


/home/gpu/pyenv/py3.8env/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 1s 4ms/step - loss: 5.0080 - val_loss: 2.8134
Epoch 2/10
242/242 [==============================] - 1s 3ms/step - loss: 1.9439 - val_loss: 1.2843
Epoch 3/10
242/242 [==============================] - 1s 3ms/step - loss: 1.0338 - val_loss: 0.8021
Epoch 4/10
242/242 [==============================] - 1s 4ms/step - loss: 0.7456 - val_loss: 0.6417
Epoch 5/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6504 - val_loss: 0.5861
Epoch 6/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6158 - val_loss: 0.5628
Epoch 7/10
242/242 [==============================] - 1s 3ms/step - loss: 0.6009 - val_loss: 0.5519
Epoch 8/10
242/242 [==============================] - 1s 3ms/step - loss: 0.5930 - val_loss: 0.5448
Epoch 9/10
242/242 [==============================] - 1s 3ms/step - loss: 0.5872 - val_loss: 0.5395
Epoch 10/10
121/121 [==============================] - 0s 2ms/step - loss: 0.5603
Epoch 1/10


...

/home/gpu/pyenv/py3.8env/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 2s 5ms/step - loss: 1.5310 - val_loss: 0.6552
Epoch 2/10
242/242 [==============================] - 1s 5ms/step - loss: 0.6411 - val_loss: 0.5410
Epoch 3/10
242/242 [==============================] - 1s 4ms/step - loss: 0.5577 - val_loss: 0.4949
Epoch 4/10
242/242 [==============================] - 1s 4ms/step - loss: 0.5187 - val_loss: 0.4684
Epoch 5/10
242/242 [==============================] - 1s 4ms/step - loss: 0.4914 - val_loss: 0.4476
Epoch 6/10
242/242 [==============================] - 1s 5ms/step - loss: 0.4719 - val_loss: 0.4347
Epoch 7/10
242/242 [==============================] - 1s 4ms/step - loss: 0.4575 - val_loss: 0.4230
Epoch 8/10
242/242 [==============================] - 1s 4ms/step - loss: 0.4462 - val_loss: 0.4150
Epoch 9/10
242/242 [==============================] - 1s 5ms/step - loss: 0.4362 - val_loss: 0.4081
Epoch 10/10
121/121 [==============================] - 0s 2ms/step - loss: 0.4293
Epoch 1/10


/home/gpu/pyenv/py3.8env/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


242/242 [==============================] - 2s 5ms/step - loss: 1.5265 - val_loss: 0.7127
Epoch 2/10
242/242 [==============================] - 1s 4ms/step - loss: 0.6849 - val_loss: 0.6082
Epoch 3/10
242/242 [==============================] - 1s 4ms/step - loss: 0.6103 - val_loss: 0.5625
Epoch 4/10
242/242 [==============================] - 1s 5ms/step - loss: 0.5549 - val_loss: 0.5013
Epoch 5/10
242/242 [==============================] - 1s 4ms/step - loss: 0.5189 - val_loss: 0.4791
Epoch 6/10
242/242 [==============================] - 1s 4ms/step - loss: 0.4965 - val_loss: 0.4645
Epoch 7/10
242/242 [==============================] - 1s 5ms/step - loss: 0.4788 - val_loss: 0.4355
Epoch 8/10
242/242 [==============================] - 1s 5ms/step - loss: 0.4686 - val_loss: 0.4274
Epoch 9/10
242/242 [==============================] - 1s 5ms/step - loss: 0.4521 - val_loss: 0.4182
Epoch 10/10
121/121 [==============================] - 0s 2ms/step - loss: 0.4463
Epoch 1/10
363/363 [=========

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f57ac3f1370>,
                   param_distributions={'lr_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f575809ea00>,
                                        'n_hidden': [0, 1, 2, 3, 4, 5],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [42]:
rnd_search_cv.best_params_

{'lr_rate': 0.0025036834641111536, 'n_hidden': 3, 'n_neurons': 64}

In [43]:
rnd_search_cv.best_score_

-0.4459107220172882

In [44]:
model = rnd_search_cv.best_estimator_.model

In [45]:
model.predict(X_test[:3])

array([[2.1433363],
       [3.9937038],
       [2.4136534]], dtype=float32)

In [46]:
Y_test[:3]

array([1.992, 3.609, 1.289])